<a href="https://colab.research.google.com/github/Santiago-R/aupa.ai/blob/main/lm-hackers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [A hacker's guide to Language Models](https://colab.research.google.com/github/fastai/lm-hackers/blob/main/lm-hackers.ipynb#scrollTo=0b017bfc-5be0-4e41-9fa1-9f685c3b0de5)


## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')  # , force_remount=True)

Mounted at /content/drive


In [2]:
# import tokenize
# from io import BytesIO

In [3]:
# from pathlib import Path
# path = Path("/content/drive/MyDrive/LLM")

## The OpenAI API

In [4]:
# Load OpenAI api key as environvent variable (from Drive's api_keys.env)
!pip install python-dotenv -qq
from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/LLM/api_keys.env')

True

In [5]:
!pip install openai -qq
from openai import ChatCompletion, Completion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00


In [6]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies whenever possible."
question = "What is money?"

c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": question}])

- [Model options](https://platform.openai.com/docs/models)

In [7]:
def response(c):
    try:
        return c['choices'][0]['message']['content']
    except KeyError:
        return c['choices'][0]['text']

In [8]:
response(c)

"Well mate, money is like the engine oil of the economy. It's what keeps things running smoothly. You see, money is a medium of exchange. It's those pieces of paper or digital numbers that we use to trade and buy stuff. Without money, we'd be stuck in a bartering system where you'd have to trade your vegemite for a can of beer or a freeze pop for a surfboard. Money makes it a whole lot easier to buy the things we want and need. It's like the fuel for our economic engine."

In [9]:
print(c.usage)

{
  "prompt_tokens": 31,
  "completion_tokens": 111,
  "total_tokens": 142
}


In [10]:
0.002 / 1000 * 150  # GPT 3.5

0.0003

In [11]:
0.03 / 1000 * 150  # GPT 4

0.0045

In [12]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"},
              {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
              {"role": "user", "content": "Really? In what way?"}])

In [13]:
response(c)

"Ah, glad you asked! Money, just like our hopping marsupials, plays a vital role in our Aussie ecosystem. Just as kangaroos are a means of exchange for nutrients and survival, money is a means of exchange for goods and services. It's the medium we use to leap from one transaction to another, helping us trade and procure what we need to survive in this wild world of ours. So, without money, it's like trying to hop around without kangaroos - it just won't work!"

In [14]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [15]:
response(askgpt('What is the meaning of life?', system=aussie_sys))

"Well mate, the meaning of life is a bit like catchin' a big wave. It's a subjective thing, ya know? Everyone's got their own idea of what gives their life purpose and fulfillment. For some, it might be findin' true love and startin' a family. For others, it might be chasin' their dreams and achievin' success in their chosen career. And for some, it might be embracin' a life of adventure and exploration. It's all about findin' what makes you feel alive and givin' it all ya got. So, go ahead and ride that wave, mate. Find what truly floats your boat and live your life with a true blue Aussie spirit!"

- [Limits](https://platform.openai.com/docs/guides/rate-limits/what-are-the-rate-limits-for-our-api)

Created by Bing:

In [16]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return ChatCompletion.create(model=model, messages=msgs)
    except openai.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [17]:
response(call_api("What's the world's funniest joke? Has there ever been any scientific analysis?"))

'The world\'s funniest joke is subjective and can vary greatly from person to person based on their sense of humor. However, a study conducted in 2002 by psychologist Richard Wiseman aimed to find the funniest joke. The study involved over 40,000 participants from 70 countries, and the winning joke was as follows:\n\n"Two hunters are out in the woods when one of them collapses. He doesn\'t seem to be breathing, and his eyes are glazed. The other guy whips out his phone and calls emergency services. He gasps, \'My friend is dead! What can I do?\' The operator says, \'Calm down, I can help. First, let\'s make sure he\'s dead.\' There is a silence, then a gunshot is heard. Back on the phone, the guy says, \'Okay, now what?\'"\n\nIt\'s worth noting that humor is highly subjective, and what one person finds hilarious might not resonate with someone else.'

In [18]:
c = Completion.create(prompt="Australian Jeremy Howard is ",
                      model="gpt-3.5-turbo-instruct", echo=True, logprobs=5)

In [19]:
response(c)

"Australian Jeremy Howard is 22 and has the world at his feet.\n\nHe's is the president of fast"

## Create our own code interpreter

In [20]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

#### Example with sums function

In [21]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [22]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [23]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'title': 'Input for `sums`',
  'type': 'object',
  'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'default': 1, 'type': 'integer'}},
  'required': ['a']}}

In [24]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [25]:
c

<OpenAIObject chat.completion id=chatcmpl-84ewQ66XpkoL750G8XrQ9kKnGqEuv at 0x7b3caa15b880> JSON: {
  "id": "chatcmpl-84ewQ66XpkoL750G8XrQ9kKnGqEuv",
  "object": "chat.completion",
  "created": 1696121382,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "sums",
          "arguments": "{ \n  \"a\": 6,\n  \"b\": 3\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 23,
    "total_tokens": 106
  }
}

In [26]:
m = c.choices[0].message
m

<OpenAIObject at 0x7b3caa728770> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{ \n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

In [27]:
k = m.function_call.arguments
print(k)

{ 
  "a": 6,
  "b": 3
}


In [28]:
funcs_ok = {'sums', 'python'}

In [29]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [30]:
call_func(c)

9

In [31]:
import ast

def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None

    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [32]:
run("""
a=1
b=2
a+b
""")

3

In [33]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [34]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [35]:
call_func(c)

KeyboardInterrupt: ignored

#### Using Python's exec & eval

In [36]:
exec_schema = {'name': 'exec',
 'description': 'Execute the given source Python code',
 'parameters': {'title': 'Input for `exec`',
  'type': 'object',
  'properties': {'source': {'title': 'S', 'type': 'string'}},
  'required': ['source']}}

In [37]:
def code_response(c, repl=True):
    txt_out = response(c)
    if txt_out == None: txt_out = ''
    if 'function_call' not in c['choices'][0]['message'].keys():
        print(txt_out)
        return  # No code output
    code = c['choices'][0]['message']['function_call']['arguments']
    if code[0] == '{': code = json.loads(code)['source']
    txt_out += f'\n==========\n{code}\n==========\n>>> '
    if repl:
        code_body = '\n'.join(code.split('\n')[:-1])
        code_footer = code.split('\n')[-1]
        exec(code_body, locals())
        result = eval(code_footer, locals())
        txt_out += str(result)
    else:
        exec(code, locals())
        result = None
    print(txt_out)
    return result

In [38]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [39]:
factorial_result = code_response(c, repl=True)


import math

factorial = math.factorial(12)
factorial
>>> 479001600


In [40]:
factorial_result

479001600

###### Using result in later calls

In [41]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[exec_schema],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "exec", "content": str(factorial_result)}])

In [42]:
print(response(c))

The factorial of 12, denoted as 12!, is equal to 479,001,600.


###### And we didn't break its basic use!

In [43]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [44]:
code_response(c)

The capital of France is Paris.


## PyTorch and Huggingface

In [48]:
!pip install transformers -qq
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

- [HF leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- [fasteval](https://fasteval.github.io/FastEval/)

In [46]:
mn = "meta-llama/Llama-2-7b-hf"

In [47]:
# Requires Huggingface login
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

OSError: ignored

In [ ]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [ ]:
toks

In [ ]:
tokr.batch_decode(toks['input_ids'])

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
tokr.batch_decode(res)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
mn = 'TheBloke/Llama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [ ]:
gen(prompt, 50)

[StableBeluga-7B](https://huggingface.co/stabilityai/StableBeluga-7B)

In [ ]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [ ]:
ques = "Who is Jeremy Howard?"

In [ ]:
gen(mk_prompt(ques), 150)

[OpenOrca/Platypus 2](https://huggingface.co/Open-Orca/OpenOrca-Platypus2-13B)

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques), 150)

In [ ]:
!pip install wikipedia-api
from wikipediaapi import Wikipedia

In [ ]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [ ]:
print(jh_page[:500])

In [ ]:
len(jh_page.split())

In [ ]:
# def mk_prompt_context(question, context):
#     return f"""Answer the question with the help of the provided context.\n\n## Context\n\n{context}\n\n## Question\n\n{question}"""

In [ ]:
res = gen(mk_prompt(ques_ctx), 300)

In [ ]:
print(res[0].split('### Assistant:\n')[1])

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
jh = jh_page.split('\n\n')[0]
print(jh)

In [ ]:
tb_page = wiki.page('Tony_Blair').text.split('\nReferences\n')[0]

In [ ]:
tb = tb_page.split('\n\n')[0]
print(tb[:380])

In [ ]:
q_emb,jh_emb,tb_emb = emb_model.encode([ques,jh,tb], convert_to_tensor=True)

In [ ]:
tb_emb.shape

In [ ]:
import torch.nn.functional as F

In [ ]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

In [ ]:
F.cosine_similarity(q_emb, tb_emb, dim=0)

### Private GPTs

- [Sooo many](https://github.com/h2oai/h2ogpt/blob/main/docs/README_LangChain.md#what-is-h2ogpts-langchain-integration-like)

## Fine tuning

In [ ]:
import datasets

[knowrohit07/know_sql](https://huggingface.co/datasets/knowrohit07/know_sql)

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 78562
    })
})

In [ ]:
trn = ds['train']
trn[3]

{'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)',
 'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'",
 'question': 'What are the hosts of competitions whose theme is not "Aliens"?'}

`accelerate launch -m axolotl.cli.train sql.yml`

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

{'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)',
 'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'",
 'question': 'Get the count of competition hosts by theme.'}

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

In [ ]:
print(sql_prompt(tst))

SYSTEM: Use the following contextual information to concisely answer the question.

USER: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
===
List all competition hosts sorted in ascending order.
ASSISTANT:


In [ ]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
ax_model = '/home/jhoward/git/ext/axolotl/qlora-out'

In [ ]:
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [ ]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',
                                             torch_dtype=torch.bfloat16, device_map=0)
model = PeftModel.from_pretrained(model, ax_model)
model = model.merge_and_unload()
model.save_pretrained('sql-model')

In [ ]:
toks = tokr(sql_prompt(tst), return_tensors="pt")

In [ ]:
res = model.generate(**toks.to("cuda"), max_new_tokens=250).to('cpu')

In [ ]:
print(tokr.batch_decode(res)[0])

<s> SYSTEM: Use the following contextual information to concisely answer the question.

USER: CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)
===
Get the count of competition hosts by theme.
ASSISTANT: SELECT COUNT(Hosts), Theme FROM farm_competition GROUP BY Theme</s>


## [llama.cpp](https://github.com/abetlen/llama-cpp-python)

[TheBloke/Llama-2-7b-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF)

In [ ]:
!pip install llama_cpp_python -qq
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path="content/llamacpp/llama-2-7b-chat.Q4_K_M.gguf")

ValueError: ignored

In [ ]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)


llama_print_timings:        load time =   192.25 ms
llama_print_timings:      sample time =    14.98 ms /    32 runs   (    0.47 ms per token,  2135.75 tokens per second)
llama_print_timings: prompt eval time =   192.16 ms /    15 tokens (   12.81 ms per token,    78.06 tokens per second)
llama_print_timings:        eval time =   767.74 ms /    31 runs   (   24.77 ms per token,    40.38 tokens per second)
llama_print_timings:       total time =  1032.79 ms


In [ ]:
print(output['choices'])

[{'text': 'Q: Name the planets in the solar system? A: 1. Pluto (no longer considered a planet) 2. Mercury 3. Venus 4. Earth 5. Mars 6.', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]
